## Data Crawling
- 사이트별로 직무내용이 너무 상이함, 구분을 지어주기가 쉽지 않음  
-> 학습에 의해 판단되므로 직무내용을 굳이 안넣어도 될 것 같음  
-> ```회사소개, 주요업무 column 추가``` 아래의 내용과 같이 모든 column 통일  
```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [2]:
import pandas as pd
import numpy as np
import re

### 원티드 데이터 전처리
- column확인 후 필요없는 column제거 후 정해진 frame의 column name으로 변경
- 정해진 frame의 column_name으로 변경
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [3]:
wanted = pd.read_csv('../Crawling/sample_data/wanted.csv')
wanted.head(1)

,직무명,채용공고명,해시태그,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택 ・ 툴,근무지역,회사명,회사분야,출처
0,웹 개발자,Python Developer (백엔드 주니어),#퇴사율5%이하#51~300명#설립4~9년#육아휴직#출산휴가#스톡옵션#건강검진#교육...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 알고리즘 자동화 설계 및 개발\r\n• 세무팀과 커뮤니케이션\r\n• AW...,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• MSA 기반의 시스템 개발 경험이 있으신 분\r\n• AWS 를 활용한 개발, ...",# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,Github\r\nJavaScript\r\nTypeScript\r\nReact.js...,서울.한국,혜움랩스,"IT, 컨텐츠",https://www.wanted.co.kr/wd/167436


- **결측치 확인 부분**  
상황에 따라 다르게 작성

In [4]:
#우대사항, 자격요건, 혜택 및 복지 부분 nan값 row 제거

wanted.dropna(subset=['우대사항', '자격요건','혜택 및 복지'], inplace=True)
wanted.count()

직무명         7874
채용공고명       7858
해시태그        7874
회사소개        7874
주요업무        7874
자격요건        7874
우대사항        7874
혜택 및 복지     7874
기술스택 ・ 툴    5695
근무지역        7874
회사명         7874
회사분야        7874
출처          7874
dtype: int64

In [7]:
#채용공고명만 비어있는 부분 채워줌, 크롤링 진행시 누락된 것으로 확인

null_data = wanted[wanted['채용공고명'].isnull() == True]['출처'].tolist()
add_data = ['[제주] 백엔드 개발자 (5년 이하)',
            '백엔드 개발자',
            '[광고 제품팀] 풀 스택 엔지니어 리드',
            'Sr. Back-end Engineer',
            'iOS 코어 개발자 (5년 이상)',
            '모바일 SW 개발자(C,C++)',
            '디바이스 부문 개발자',
            'Unity 개발자',
            '기술연구소 백엔드 서버 개발자',
            '백엔드 개발자 (BackEnd Engineer)',
            'Senior Software Engineer, Android',
            '백엔드 개발자(Node.js, 시니어)',
            '백엔드 개발자 (Node.js, 주니어)',
            0,
            'AI SW 개발자',
            'Data Engineer(2년 이상)']

for n_data, a_data in zip(null_data,add_data):
    if a_data == 0:
        wanted.drop(wanted[wanted['출처'] == n_data].index, inplace=True)
        continue
    wanted.loc[wanted['출처'] == n_data, '채용공고명'] = a_data

#drop으로 인한 index 누락 reindexing
wanted.reset_index(drop=True, inplace=True)
wanted.count()

직무명         7873
채용공고명       7873
해시태그        7873
회사소개        7873
주요업무        7873
자격요건        7873
우대사항        7873
혜택 및 복지     7873
기술스택 ・ 툴    5695
근무지역        7873
회사명         7873
회사분야        7873
출처          7873
dtype: int64

In [8]:
wanted.rename(columns={'직무명':'직무내용',
                       '채용공고명':'공고명',
                       '기술스택 ・ 툴':'기술 스택',
                       '혜택 및 복지':'복지',
                       '출처':'출처 URL',
                       '우대사항':'우대조건',
                       }, inplace=True)

wanted.columns

Index(['직무내용', '공고명', '해시태그', '회사소개', '주요업무', '자격요건', '우대조건', '복지', '기술 스택',
       '근무지역', '회사명', '회사분야', '출처 URL'],
      dtype='object')

- 지원기간은 이야기 해봐야 할 듯, 경력조건은 자격요건의 내용에서 추출

### wanted 패턴
- 경력 n년 이상
- 경력 n년이상
- n~k년
- n년~k년

In [43]:
def extract_years(string, patterns):
    for pattern in patterns:
        match = re.search(pattern, string)
        if match:
            if '~' in pattern:
                start_year = match.group(1)
                return int(start_year)
            else:
                return int(match.group(1))
    
    return 0


In [10]:
max_count = wanted['자격요건'].count()
wanted['경력조건'] = None
patterns = [r'(\d+)년 이상', r'(\d+)년이상', r'(\d+)~(\d+)년', r'(\d+)년~(\d+)년']

for i in range(0, max_count):
    wanted.loc[i, '경력조건'] = extract_years(wanted['자격요건'][i], patterns)


In [11]:
wanted['경력조건'].unique()

array([2, 0, 3, 1, 7, 6, 4, 5, 8, 10, 15, 9, 12], dtype=object)

- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [14]:
new_order = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지', '회사소개', '주요업무']
wanted = wanted.reindex(columns=new_order)
wanted.count()

회사명       7873
지원기간         0
공고명       7873
직무내용      7873
자격요건      7873
우대조건      7873
경력조건      7873
기술 스택     5695
출처 URL    7873
복지        7873
회사소개      7873
주요업무      7873
dtype: int64

In [15]:
#csv 파일로 저장

wanted.to_csv('./Data_result/pre_wanted.csv',index=False)

### 잡플래닛 데이터 전처리
- column확인 후 필요없는 column제거 후 정해진 frame의 column name으로 변경
- 정해진 frame의 column_name으로 변경
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [49]:
jobplanet = pd.read_csv('../Crawling/sample_data/jobplanet.csv')
jobplanet.head(1)

,href,title,company_name,job_location,job_classification,due_date,work_experience,skills,company_intro,main_task,qualification,preference,procedure,benefits,job_location_detail,detail_description
0,https://www.jobplanet.co.kr/job/search?posting...,DB 관리 (카드IS 3팀),롯데정보통신(주),서울,데이터 엔지니어,2023.07.03 D-7,3년 이상,"SQL, MYSQL, DB, 데이터베이스","오늘을 새롭게, 내일을 이롭게. 롯데\r\n\r\n롯데와 함께 비전을 공유할 수 있...",ㆍSQL 튜닝\r\nㆍDatabase Admin\r\nㆍDB Performance 튜닝,ㆍ경력 3년 이상\r\nㆍOracle Database 운영/백업/모니터링/장애처리 ...,ㆍSQL 튜닝\r\nㆍOGG 솔루션 운영 경험\r\nㆍ금융권 대용량 DB시스템 운영...,지원서접수 > 서류전형 > 직무테스트/인성진단(온라인) > 면접전형 > 평판조회 >...,ㆍ복지포인트 및 경조지원\r\nㆍ건강검진\r\nㆍ학자금/교육지원 등 다양한 복리후생...,서울 금천구 가산동 533-2 롯데센터 15층,"<section class=""recruitment-detail""><div class..."


In [50]:
jobplanet.rename(columns={'href': '출처 URL', 
                          'title': '공고명',
                          'company_name':'회사명',
                          'job_classification':'직무내용',
                          'due_date':'지원기간',
                          'work_experience':'경력조건',
                          'skills':'기술 스택',
                          'main_task':'주요 업무',
                          'qualification':'자격요건',
                          'preference':'우대조건',
                          'benefits':'복지',
                          'company_intro': '회사소개',
                          'main_task':'주요업무'
                          }, inplace=True)
jobplanet.columns

Index(['출처 URL', '공고명', '회사명', 'job_location', '직무내용', '지원기간', '경력조건', '기술 스택',
       '회사소개', '주요업무', '자격요건', '우대조건', 'procedure', '복지',
       'job_location_detail', 'detail_description'],
      dtype='object')

In [51]:
new_order = ['회사명', '지원기간', '공고명', '직무내용', '자격요건', '우대조건', '경력조건', '기술 스택', '출처 URL', '복지','회사소개','주요업무']
jobplanet = jobplanet.reindex(columns=new_order)
jobplanet.head(1)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지,회사소개,주요업무
0,롯데정보통신(주),2023.07.03 D-7,DB 관리 (카드IS 3팀),데이터 엔지니어,ㆍ경력 3년 이상\r\nㆍOracle Database 운영/백업/모니터링/장애처리 ...,ㆍSQL 튜닝\r\nㆍOGG 솔루션 운영 경험\r\nㆍ금융권 대용량 DB시스템 운영...,3년 이상,"SQL, MYSQL, DB, 데이터베이스",https://www.jobplanet.co.kr/job/search?posting...,ㆍ복지포인트 및 경조지원\r\nㆍ건강검진\r\nㆍ학자금/교육지원 등 다양한 복리후생...,"오늘을 새롭게, 내일을 이롭게. 롯데\r\n\r\n롯데와 함께 비전을 공유할 수 있...",ㆍSQL 튜닝\r\nㆍDatabase Admin\r\nㆍDB Performance 튜닝


- **결측치 확인 부분**  
```count()로 결측치 부분 확인 후 삭제 or 채움 결정```  
-> 데이터가 많지 않다면 삭제하는 방향

In [52]:
jobplanet.count()

회사명       108
지원기간      108
공고명       108
직무내용      108
자격요건      108
우대조건      106
경력조건      108
기술 스택     108
출처 URL    108
복지        107
회사소개      108
주요업무      108
dtype: int64

In [53]:
# 결측치의 내용에 따라 변경 
jobplanet.drop(jobplanet[jobplanet['복지'].isnull() == True].index, inplace=True)
jobplanet.drop(jobplanet[jobplanet['우대조건'].isnull() == True].index, inplace=True)
jobplanet.reset_index(drop=True, inplace=True)
jobplanet.count()

회사명       105
지원기간      105
공고명       105
직무내용      105
자격요건      105
우대조건      105
경력조건      105
기술 스택     105
출처 URL    105
복지        105
회사소개      105
주요업무      105
dtype: int64

- 경력조건 int형 변환
- wanted data 처리 때 만들었던 extract_years()를 사용한다.

In [ ]:
jobplanet['경력조건']

In [ ]:
max_count = jobplanet['경력조건'].count()
patterns = [r'(\d+)년 이상']

for i in range(0, max_count):
    jobplanet.loc[i, '경력조건'] = extract_years(jobplanet['경력조건'][i], patterns)

jobplanet['경력조건'].unique()

In [57]:
jobplanet['경력조건'].unique()

array([3, 0, 2, 4, 10, 6, 8, 5, 1, 7], dtype=object)

- 지원기간 ```YYYY.MM.DD``` 형태로 변환

In [58]:
jobplanet['지원기간'] = jobplanet['지원기간'].str.split('\xa0').str[0]
jobplanet['지원기간'].unique()

array(['2023.07.03', '2023.06.26', '2023.07.21', '2023.06.30',
       '2023.09.20', '2023.09.26', '2023.09.19', '2023.07.14',
       '2023.09.15', '2023.09.14', '2023.09.13', '2023.09.09',
       '2023.07.09', '2023.09.07', '2023.07.04', '2023.08.31',
       '2023.08.25', '2023.08.26', '2023.08.24', '2023.08.23',
       '2023.08.29', '2023.08.19', '2023.08.18', '2023.08.17',
       '2023.08.16', '2023.08.21', '2023.08.09', '2023.08.22',
       '2023.08.27', '2023.08.15', '2023.07.12', '2023.10.29',
       '2023.07.17', '2023.08.02', '2023.07.11', '2023.09.24',
       '2023.09.02', '2023.09.06', '2023.07.13', '2023.07.28',
       '2023.07.18', '2023.07.19', '2023.09.21', '2023.09.22',
       '2023.07.01', '2023.08.10', '2023.09.11', '2023.08.14',
       '2023.07.31', '2023.09.27'], dtype=object)

In [59]:
jobplanet.count()

회사명       105
지원기간      105
공고명       105
직무내용      105
자격요건      105
우대조건      105
경력조건      105
기술 스택     105
출처 URL    105
복지        105
회사소개      105
주요업무      105
dtype: int64

In [60]:
#csv 파일로 저장

jobplanet.to_csv('./Data_result/pre_jobplanet.csv',index=False)

### 점핏 데이터 전처리

- 크롤링 문제 확인으로 직무내용 별로 csv 파일 생성 후 concat
- 지원기간과 경력조건 data만 맞추기  
-> 기존 column에서 회사소개, 주요업무 column 추가
- ```회사명, 지원기간, 공고명, 직무내용, 자격요건, 우대조건, 경력조건, 기술 스택, 출처 URL, 복지, 회사소개, 주요업무```

In [93]:
jumpit = pd.DataFrame()

for i in range(1, 22):
    if i == 14:
        continue
    df = pd.read_csv(f'../Crawling/jumpit_data/크롤링_점핏_{i}.csv')
    jumpit = pd.concat([jumpit, df])

jumpit.reset_index(drop=True, inplace=True)

In [94]:
jumpit.head(1)

,회사명,지원기간,공고명,직무내용,자격요건,우대조건,경력조건,기술 스택,출처 URL,복지,회사소개,주요업무
0,"미디어로그는 MVNO(U+유모바일)사업, 중고폰사업(셀로), PP(방송채널)사업과 ...",마감일\n상시,미디어로그(LG그룹) 웹 하드 3.0 SM 경력 채용,서버/백엔드 개발자,"• Java, Spring, Spring Boot, JSP 개발 스킬 필수\n• O...","• LG U+ SI/SM 업무 경험\n• 서버 간 연동 처리 경험 (restful,...",경력\n경력 10~15년,Java\nJSP\nSpring Boot\nMariaDB\nOracle\nLinux,https://www.jumpit.co.kr/position/16607,• 통신비 지원\n- U+유모바일 음성무제한 요금제 지원\n\n• 자녀 학비 지원\...,"미디어로그는 MVNO(U+유모바일)사업, 중고폰사업(셀로), PP(방송채널)사업과 ...",• 웹 하드 3.0 운영 유지보수 기능 개발\n• VOC 대응\n• 장애 대응 및 ...


In [95]:
#크롤링 과정에서 결측치는 없게 처리함

jumpit.count()

회사명       3187
지원기간      3187
공고명       3187
직무내용      3187
자격요건      3187
우대조건      3187
경력조건      3187
기술 스택     3187
출처 URL    3187
복지        3187
회사소개      3187
주요업무      3187
dtype: int64

In [96]:
#마감일\n 삭제와 '-' -> '.' replace

jumpit['지원기간'] = jumpit['지원기간'].str.replace('마감일\n', '').str.replace('-','.')
jumpit['지원기간'].unique()

array(['상시', '2023.07.31', '2023.12.31', '2023.08.31', '2023.08.03',
       '2023.08.07', '2023.08.16', '2023.08.01', '2023.07.21',
       '2023.07.28', '2023.07.23', '2023.11.30', '2023.07.24',
       '2023.08.04', '2023.07.26', '2023.08.02', '2023.07.30',
       '2023.09.07', '2023.07.29', '2023.09.30', '2023.08.11',
       '2023.09.13', '2023.08.21', '2023.09.12', '2023.08.06',
       '2023.10.09', '2023.08.12', '2023.07.27', '2023.10.03',
       '2023.08.23', '2023.08.22', '2023.08.18', '2024.08.14',
       '2023.09.08', '2023.08.17', '2023.08.15', '2023.08.09',
       '2023.10.17', '2023.10.18', '2023.09.11', '2023.08.08',
       '2023.08.28', '2023.08.10', '2023.07.22', '2023.08.13',
       '2023.10.02', '2023.09.29', '2023.10.19', '2023.10.06',
       '2023.09.27', '2023.10.16', '2023.12.01', '2023.08.05',
       '2023.09.06'], dtype=object)

- 경력 조건 부분 ```경력\n``` 제거
- 신입~n년 부분 1~n 년으로 변경
- 경력조건 int형 변환
- wanted data 처리 때 만들었던 extract_years()를 사용한다.

In [100]:
jumpit['경력조건'].unique()

array(['경력\n경력 10~15년', '경력\n경력 3~10년', '경력\n경력 3~6년', '경력\n경력 5~10년',
       '경력\n경력 2~10년', '경력\n경력 1~3년', '경력\n신입~10년', '경력\n경력 2~6년',
       '경력\n경력 3~20년', '경력\n경력 3~12년', '경력\n경력 3~7년', '경력\n경력 1~7년',
       '경력\n경력 3~15년', '경력\n경력 10~20년', '경력\n경력 5~12년', '경력\n경력 5~20년',
       '경력\n신입~3년', '경력\n경력 4~15년', '경력\n신입~2년', '경력\n경력 2~20년',
       '경력\n경력 5~15년', '경력\n경력 4~6년', '경력\n경력 2~7년', '경력\n경력 1~5년',
       '경력\n경력 3~5년', '경력\n경력 2~9년', '경력\n신입~20년', '경력\n경력 7~10년',
       '경력\n경력 7~15년', '경력\n경력 1~10년', '경력\n경력 2~15년', '경력\n신입~15년',
       '경력\n경력 6~20년', '경력\n경력 4~10년', '경력\n경력 1~15년', '경력\n경력 6~15년',
       '경력\n신입~5년', '경력\n경력 2~5년', '경력\n경력 3~8년', '경력\n경력 5~8년',
       '경력\n경력 6~16년', '경력\n경력 6~8년', '경력\n경력 4~11년', '경력\n경력 2~12년',
       '경력\n경력 2~8년', '경력\n경력 8~15년', '경력\n경력 4~7년', '경력\n경력 12~16년',
       '경력\n경력 2~3년', '경력\n경력 5~7년', '경력\n경력 3~9년', '경력\n경력 1~8년',
       '경력\n경력 7~20년', '경력\n경력 10~14년', '경력\n경력 3~4년', '경력\n경력 7~25년',
       '경력\n경력 8~13년', '경력\n경력 4~20년',

In [101]:
max_count = jumpit['경력조건'].count()
jumpit['경력조건'] = jumpit['경력조건'].str.replace('신입~', '0~')
patterns = [r'경력\n경력 (\d+)~(\d+)년',r'경력\n(\d+)~(\d+)년']

for i in range(0, max_count):
    jumpit.loc[i, '경력조건'] = extract_years(jumpit['경력조건'][i], patterns)
    
jumpit['경력조건'].unique()

array([10, 3, 5, 2, 1, 0, 4, 7, 6, 8, 12, 9, 17, 11, 15], dtype=object)

In [102]:
jumpit.to_csv('../Crawling/sample_data/jumpit.csv', index=False)
jumpit.to_csv('./Data_result/pre_jumpit.csv', index=False)

## CSV파일 concat 

- 기술스택, 자격요건 column -> ```자격요건 column으로 concat```
- 현재 코드에서 ```Crawling/sample_data/```에 있는 csv 파일을 통해 ```Data_Pretreatment/Data_result/result.csv```를 만듬

In [149]:
wanted = pd.read_csv('./Data_result/pre_wanted.csv')
jobplanet = pd.read_csv('./Data_result/pre_jobplanet.csv')
jumpit_ = pd.read_csv('./Data_result/pre_jumpit.csv')

In [150]:
result = pd.concat([wanted, jobplanet, jumpit], axis=0)
result.reset_index(drop=True, inplace=True)
result.count()

회사명       11165
지원기간       3292
공고명       11165
직무내용      11165
자격요건      11165
우대조건      11165
경력조건      11165
기술 스택      8987
출처 URL    11165
복지        11165
회사소개      11165
주요업무      11165
dtype: int64

- 자격요건, 기술 스택 concat
- '자격요건'과 '기술 스택' column의 type을 ```object -> str```변환 후 join을 통해 concat

In [151]:
result['자격요건'] = result['자격요건'].astype(str)
result['기술 스택'] = result['기술 스택'].astype(str)
concat_df = pd.DataFrame({'자격요건': [" ".join(row) for row in zip(result['자격요건'], result['기술 스택'])]})

result['자격요건'] = concat_df['자격요건'][0]
result.drop(['기술 스택', '지원기간'], axis=1, inplace=True)

In [152]:
result.head(1)

,회사명,공고명,직무내용,자격요건,우대조건,경력조건,출처 URL,복지,회사소개,주요업무
0,혜움랩스,Python Developer (백엔드 주니어),웹 개발자,"• 최소 2년 이상 혹은 그에 준하는 해당 업무 경험\r\n• Python(필수),...","• MSA 기반의 시스템 개발 경험이 있으신 분\r\n• AWS 를 활용한 개발, ...",2,https://www.wanted.co.kr/wd/167436,# 몰입을 위한\r\n• 액면가 스톡옵션의 기회가 열려있습니다. (2023년 7월 ...,"- 간단소개\r\n혜움랩스는 세무,재무,경리 등 AI기반 비서형 금융 서비스를 Fa...",• 세무 알고리즘 자동화 설계 및 개발\r\n• 세무팀과 커뮤니케이션\r\n• AW...


In [154]:
result.count()

회사명       11165
공고명       11165
직무내용      11165
자격요건      11165
우대조건      11165
경력조건      11165
출처 URL    11165
복지        11165
회사소개      11165
주요업무      11165
dtype: int64

---------------------------------------------------

## 최종 result 데이터 저장


In [153]:
result.to_csv('./Data_result/pre_result.csv', index=False)